In [1]:
import pipeline_models as pl
import pandas as pd
import numpy as np
import os

from IPython.display import display, HTML
from IPython.display import clear_output

In [3]:
all_files=[]
all_files_name = []
for root, dirs, files in os.walk("./data/"):
    for file in files:
        if file.endswith(".csv"):
            all_files.append(os.path.join(root, file))
            all_files_name.append(root.split("/")[-1] + "_" + file.replace(".", "_"))
# df = pd.read_csv()
for i in range(1,30):
    df = pd.read_csv(all_files[i], 
                sep=';', 
                index_col='datetime', 
                parse_dates=True)

    autoencoder = pl.PipelineModel()
    autoencoder.apply_standard_scaling(())
    autoencoder.apply_nested_rotations((2, 0.75))
    autoencoder.apply_feature_bagging(())
    autoencoder.apply_model(("ae", "task_name_1"))

    autoencoder.pipeline_fit(df[:400].drop(['anomaly', 'changepoint'], axis=1))

    final_df = autoencoder.pipeline_transform(df.drop(['anomaly', 'changepoint'], axis=1))

    final_df["final_anomaly"] = (final_df["anomaly_by_autoencoder_task_name_1_score"] > (3/2) * autoencoder.model.UCL).astype(int)
    print(f'MAX: {final_df["anomaly_by_autoencoder_task_name_1_score"].max()}, {3/2*autoencoder.model.UCL}, {(3/2)*autoencoder.model.UCL}')
    print(autoencoder.model.model_name)
    print(autoencoder.model.model)
    print(autoencoder.model.UCL)
    print(autoencoder.model.anomaly_col)
    display(final_df.groupby(["final_anomaly"]).count())

In [9]:
df = pd.read_csv(all_files[1], 
                sep=';', 
                index_col='datetime', 
                parse_dates=True)
print(df.shape)
autoencoder = pl.PipelineModel()
autoencoder.apply_standard_scaling(())
autoencoder.apply_nested_rotations((2, 0.75))
autoencoder.apply_feature_bagging(())

autoencoder.apply_model(("ae", "task_name_1"))


autoencoder.pipeline_fit(df[:400].drop(['anomaly', 'changepoint'], axis=1))
a = autoencoder.pipeline_transform(df.drop(['anomaly', 'changepoint'], axis=1))
print(f"{(3/2) * autoencoder.model.UCL} - {a['anomaly_by_autoencoder_task_name_1_score'].max()}")
a["final_anomaly"] = (a['anomaly_by_autoencoder_task_name_1_score'] > (3/2)*autoencoder.model.UCL).astype(int)
print(a.shape)
a.groupby(['final_anomaly']).count()

(745, 10)
12.275505719405274 - 17.007816131835693
(745, 2)


,anomaly_by_autoencoder_task_name_1_score
final_anomaly,
0,642
1,103


In [6]:
[(col,a[col].max(), a[col].min()) for col in a.columns.tolist()]

[('anomaly_by_autoencoder_task_name_1_score',
  6.0098352468710505,
  0.4351935963220172)]